# Logistic Regression
![image](https://user-images.githubusercontent.com/28910538/63490991-00bc1780-c4f1-11e9-8963-b5ab8daeb56a.png)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
x_data = [[1,2], [2,3], [3,1], [4,3], [5,3], [6,2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [5]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [7]:
print('e^1 equals : ', torch.exp(torch.FloatTensor([1])))

e^1 equals :  tensor([2.7183])


In [8]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [10]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [13]:
print('1/(1+e^{-1}) equals : ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals :  tensor([0.7311])


In [14]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [15]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


## Computing the Cost Fcuntion
- H(X) = P(X=1;w)
    - W가 주어졌을때 X가 1이 될 확률

In [16]:
-(y_train[0] * torch.log(hypothesis[0])) + (1 - y_train[0]) * torch.log(1- hypothesis[0])

tensor([-0.6931], grad_fn=<AddBackward0>)

In [19]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1- hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [21]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


## F.binary_cross_entropy
- 간단하게 cross entropy 함수 구현

In [22]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [25]:
W = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD((W, b), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # gradient 초기화
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}/{nb_epochs} Cost : {cost}')

Epoch 0/1000 Cost : 0.6931471824645996
Epoch 100/1000 Cost : 0.134722039103508
Epoch 200/1000 Cost : 0.08064314723014832
Epoch 300/1000 Cost : 0.05790002644062042
Epoch 400/1000 Cost : 0.045299723744392395
Epoch 500/1000 Cost : 0.037260960787534714
Epoch 600/1000 Cost : 0.03167248144745827
Epoch 700/1000 Cost : 0.02755594439804554
Epoch 800/1000 Cost : 0.024394338950514793
Epoch 900/1000 Cost : 0.02188829891383648
Epoch 1000/1000 Cost : 0.019852163270115852


## Evaluation

In [27]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [28]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [31]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


## Higher Implementation with Class

In [49]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # input, output dimension
        self.liner = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.liner(x))

In [50]:
model = BinaryClassifier()

In [52]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x_train)
    
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print(f'Epoch : {epoch} / {nb_epochs} Cost : {cost.item()} accracuy : {accuracy*100}')

Epoch : 0 / 100 Cost : 1.0405453443527222 accracuy : 50.0
Epoch : 10 / 100 Cost : 0.6070860624313354 accracuy : 83.33333333333334
Epoch : 20 / 100 Cost : 0.5440798997879028 accracuy : 83.33333333333334
Epoch : 30 / 100 Cost : 0.4455174207687378 accracuy : 83.33333333333334
Epoch : 40 / 100 Cost : 0.3612867593765259 accracuy : 83.33333333333334
Epoch : 50 / 100 Cost : 0.2826939821243286 accracuy : 83.33333333333334
Epoch : 60 / 100 Cost : 0.2156440168619156 accracuy : 83.33333333333334
Epoch : 70 / 100 Cost : 0.17125405371189117 accracuy : 100.0
Epoch : 80 / 100 Cost : 0.1508191078901291 accracuy : 100.0
Epoch : 90 / 100 Cost : 0.13965849578380585 accracuy : 100.0
Epoch : 100 / 100 Cost : 0.1304633468389511 accracuy : 100.0
